In [13]:
from time import time
import networkx as nx
import pickle
import time
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import os
import torch
from collections import defaultdict
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils import clip_grad_norm
from torch.utils.data import DataLoader
import torch.utils.data as Data
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence
import math
import torch.nn.init as init
import torch.nn.functional as F
from torch.nn.parameter import Parameter
%matplotlib inline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class args(object):
    
    #### DATA  ####
    node_size = 6 # equals to the number of nodes + 1 (zero_padding) # number of colour + 1 for padding
    seq_len = 10
    
    #### Training ####
    batch_size = 32
    lr = 0.02
    beta1 = 0.9
    beta2 = 0.999
    LSTM_maxnorm = 0.25
    acc_threshold = 0.5
    try_valid = 20
    running_loss = False
    n_epoch = 50
    weight_reg = 0
    
    
    #### Model ####
    RNN_model = 'lstm' # The hidden_size has to be a list if using nalu
    hidden_size = [512]
    num_layers = 5
    embedding_dim = 256
    embedding_maxnorm = None
    bidirectional = False
    model_name = 'SiameseColourWithoutRestriction' 
    model_path ='./'+ model_name +'/Model/'
    
if not os.path.exists(args.model_path):
    os.makedirs(args.model_path)

'''
TODO:

'''   
# SimplePathWith10th5Colours
with open("SimplePathWith10th5Colours.txt", "rb") as fp:   # Unpickling
    df = pickle.load(fp)
X = df[['left','right']]     
Y = df['target']    
del df

#Seperate to training, validation, and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 64)
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size= 0.05,random_state= 64)
Y_test = Y_test.values
Y_train = Y_train.values
Y_validation = Y_validation.values

#Check shape
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

def padding(data):
    left = [] 
    for i in range(data.shape[0]):
        left.append((data.iloc[i]['left']))
    right = [] 
    for i in range(data.shape[0]):
        right.append((data.iloc[i]['right']))
    return torch.tensor(np.array([right,left])).transpose(1,0)

def plot_train_hist(train_hist, step = None, ):
    fig = plt.figure(figsize=(20, 10))
    for name in train_hist.keys():
        if 'Loss' in name:
            plt.subplot(211)
            plt.plot(train_hist[name],marker='o',label= name)
            plt.ylabel('Loss',fontsize=15)
            plt.xlabel('Number of epochs',fontsize=15)
            plt.title('Loss',fontsize=20,fontweight ="bold")
            plt.legend(loc='upper left')
        else:
            plt.subplot(212)
            plt.plot(train_hist[name],marker='o',label= name)
            plt.ylabel('Accuracy',fontsize=15)
            plt.xlabel('Number of epochs',fontsize=15)
            plt.title('Accuracy',fontsize=20,fontweight ="bold")
            plt.legend(loc='upper left')
        
    plt.tight_layout()
    plt.show()
    if step is not None:
        fig.savefig("Train_Hist"+str(step)+".png") 

def CreateDataset(X_left, X_right, Y):
    
    X_left_list = list(X_left)
    X_right_list = list(X_right)
    
    X_left_seq_len = torch.tensor(list(map(len, X_left_list)))
    X_right_seq_len = torch.tensor(list(map(len, X_right_list)))
    X_left_seq_len, X_left_len_idx = X_left_seq_len.sort(0,descending = True)
    X_right_seq_len, X_right_len_idx = X_right_seq_len.sort(0,descending = True)
    print('X_len',  X_left_seq_len , X_right_seq_len)
    X_left_ordered = [torch.LongTensor(X_left_list[i]) for i in X_left_len_idx]
    X_right_ordered = [torch.LongTensor(X_right_list[i]) for i in X_right_len_idx]
    X_left_p = pad_sequence(X_left_ordered, batch_first = True)
    X_right_p = pad_sequence(X_right_ordered, batch_first = True)
    Y = torch.FloatTensor(np.array(Y))
    train_dataset  = Data.TensorDataset(X_left_p, X_left_seq_len, X_right_p, X_right_seq_len, Y)
    
    return train_dataset


In [14]:
# # If it's not work, we can pad the seq to certain len.
# X_train_left_list = list(X_train['left'])
# X_train_right_list = list(X_train['right'])
# X_train_left_seq_len = torch.tensor(list(map(len, X_train_left_list)))
# X_train_right_seq_len = torch.tensor(list(map(len, X_train_right_list)))
# X_train_left_seq_len, X_train_left_len_idx = X_train_left_seq_len.sort(0,descending = True)
# X_train_right_seq_len, X_train_right_len_idx = X_train_right_seq_len.sort(0,descending = True)
# X_left_train_ordered = [torch.tensor(X_train_left_list[i]) for i in X_train_left_len_idx]
# X_right_train_ordered = [torch.tensor(X_train_right_list[i]) for i in X_train_right_len_idx]
# X_train_left = pad_sequence(X_left_train_ordered, batch_first = True)
# X_train_right = pad_sequence(X_right_train_ordered, batch_first = True)
# Y_train = torch.FloatTensor(np.array(Y_train))
# train_dataset  = Data.TensorDataset(X_train_left, X_train_right, Y_train)

In [15]:
# X_len = 36336
# X_train_list = list(X_train['left']) + list(X_train['right'])
# X_train_seq_len = torch.tensor(list(map(len, X_train_list)))
# X_train_seq_len, X_train_idx = X_train_seq_len.sort(0, descending= True)
# # the left and the right will be mixed after the sort operator 
# # Seperate the idx first and use the idx to choose the dordered list,
# left_idx = [ i for i in X_train_idx if i < X_len]
# right_idx = [ i for i in X_train_idx if i >= X_len]

In [16]:
train_dataset = CreateDataset(X_train['left'], X_train['right'], Y_train)
val_dataset = CreateDataset(X_validation['left'], X_validation['right'], Y_validation)
test_dataset = CreateDataset(X_test['left'], X_test['right'], Y_test)

X_len tensor([20, 20, 20,  ...,  4,  2,  2]) tensor([20, 20, 20,  ...,  2,  2,  2])
X_len tensor([20, 20, 20, 20, 20, 20, 20, 20, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16

In [17]:
print('Traninig data size: ', len(train_dataset))
print('Validation data size: ', len(val_dataset))
print('Test data size: ', len(test_dataset))

Traninig data size:  15200
Validation data size:  800
Test data size:  4000


In [18]:
# #Padding and creat the loaders
# X_train = padding(X_train)
# Y_train = torch.FloatTensor(np.array(Y_train))
# train_dataset  = Data.TensorDataset(X_train,Y_train)

# X_validation = padding(X_validation)
# Y_validation = torch.FloatTensor(np.array(Y_validation))
# val_dataset  = Data.TensorDataset(X_validation,Y_validation)

# X_test = padding(X_test)
# Y_test = torch.FloatTensor(np.array(Y_test))
# test_dataset  = Data.TensorDataset(X_test,Y_test)

In [19]:
class NeuralAccumulatorCell(nn.Module):
    
    # Feed forward but Weight decomposition
    
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim

        self.W_hat = Parameter(torch.Tensor(out_dim, in_dim))
        self.M_hat = Parameter(torch.Tensor(out_dim, in_dim))
        self.W = Parameter(torch.tanh(self.W_hat) * torch.sigmoid(self.M_hat))
        self.register_parameter('bias', None)

        init.kaiming_uniform_(self.W_hat, a=math.sqrt(5))
        init.kaiming_uniform_(self.M_hat, a=math.sqrt(5))
        
        #init.normal_(self.W_hat)
        #init.normal_(self.M_hat)
        
    def forward(self, input):
        return F.linear(input, self.W, self.bias)


class NAC(nn.Module):
    
    def __init__(self, dims):
        '''
        dims = [input_dim + hidden_dims + output_dims]
        '''
        super().__init__()
        self.num_layers = len(dims) - 1
        
        layers = nn.ModuleList()
        layers.extend([NeuralAccumulatorCell(dims[i],dims[i+1]) for i in range(self.num_layers)])
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        out = self.model(x)
        return out

class NeuralArithmeticLogicUnitCell(nn.Module):

    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.eps = 1e-10

        self.G = Parameter(torch.Tensor(out_dim, in_dim))
        self.W = Parameter(torch.Tensor(out_dim, in_dim))
        self.register_parameter('bias', None)
        self.nac = NeuralAccumulatorCell(in_dim, out_dim)

        init.kaiming_uniform_(self.G, a = math.sqrt(5))
        init.kaiming_uniform_(self.W, a = math.sqrt(5))

    def forward(self, input):
        
        a = self.nac(input)
        g = torch.sigmoid(F.linear(input, self.G, self.bias))
        add_sub = g * a
        log_input = torch.log(torch.abs(input) + self.eps)
        m = torch.exp(self.nac(log_input))
        # m = torch.exp(F.linear(log_input, self.W, self.bias))
        mul_div = (1 - g) * m
        y = add_sub + mul_div
        return y


class NALU(nn.Module):
    
    def __init__(self, dims):
        super().__init__()
        self.num_layers = len(dims) - 1
        layers = nn.ModuleList()
        layers.extend([NeuralArithmeticLogicUnitCell(dims[i],dims[i+1]) for i in range(self.num_layers)])
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        out = self.model(x)
        return out

class NALU_LSTMCell(nn.Module):

    def __init__(self, input_size, hidden_size, bias=True):
        super().__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.i2h = nn.Sequential(
            nn.Linear(input_size, 4 * hidden_size, bias=bias),
            nn.BatchNorm1d(4 * hidden_size),
#             nn.LeakyReLU(0.2,inplace=True),
#             nn.Linear(4 *input_size, 4 * hidden_size, bias=bias),
        )
        self.h2h = nn.Sequential(
            nn.Linear(hidden_size, 4 * hidden_size, bias=bias),
           nn.BatchNorm1d(4 * hidden_size),
#             nn.LeakyReLU(0.2,inplace=True),
#             nn.Linear(4 * hidden_size, 4 * hidden_size, bias=bias)
        )
        self.nalu_h = NALU([hidden_size, hidden_size])
        self.nalu_c = NALU([hidden_size, hidden_size])
        self.out = nn.Linear(hidden_size, input_size, bias=bias)
        self.apply(self.weight_init)

    def weight_init(self,m):

        std = 1.0 / math.sqrt(self.hidden_size)
        for name, w in m.named_parameters():
            w.data.uniform_(-std, std)

    def forward(self, x, hidden = None):
        
        if hidden is None:
            hidden = x.new_zeros(x.size(0), self.hidden_size, requires_grad=False)
            hidden = (hidden, hidden)
            
        h, c = hidden
        
        preact = self.i2h(x) + self.h2h(h)
        
        # First: apply nalu to replace activation func
        
        # self.nalu(preact)
        
        gates = preact[:, :3 * self.hidden_size].sigmoid()
        g_t = preact[:, 3 * self.hidden_size:].tanh()
        i_t = gates[:, :self.hidden_size] 
        f_t = gates[:, self.hidden_size:2 * self.hidden_size]
        o_t = gates[:, -self.hidden_size:]
        
        # Second: Apply it in the output and hidden layer

        c_t = (c*f_t) + (i_t*g_t)

        h_t = o_t * c_t.tanh()
        
        # return x, (h_t, c_t) # LSTM
        
        # return x + self.out(h_t), (h_t + h, c_t + c) # Residule LSTM

        return  x + self.out(h_t) , (self.nalu_h(h_t + h), self.nalu_c(c_t + c)) # Residule NALU

class NALU_LSTM(nn.Module):
    
    def __init__(self,input_size, hidden_sizes, bidirectional = False):
        
        super().__init__()
        self.bidirectional = bidirectional
        
        if self.bidirectional:
            self.num_dir = 2
        else:
            self.num_dir = 1
            
        self.input_size = input_size
        self.L = len(hidden_sizes)
        self.layers = nn.ModuleList()
        self.layers.extend([NALU_LSTMCell(input_size,i) for i in hidden_sizes])
        self.c0 = nn.ParameterList([nn.Parameter(torch.randn(self.num_dir, 1,i)) for i in hidden_sizes])
        self.h0 = nn.ParameterList([nn.Parameter(torch.randn(self.num_dir, 1,i)) for i in hidden_sizes])
        
        
    def forward(self, input):
        
        '''
        input_shape = B, S, input_size
        output_shape = B, num_dir, L, S, input_size
        hidden, cells = S * (B, hidden_dim)
        '''
        
        B,S = input.shape[:-1]
        
        outputs = torch.zeros(B, self.num_dir, self.L+1, S, self.input_size)
        outputs[:,:,0,:,:] = input.unsqueeze(1).expand_as(outputs[:,:,0,:,:])
        hiddens = []
        cells = []
    
        for i, layer in enumerate(self.layers):
            f_h, f_c = self.h0[i][0].repeat(B,1), self.c0[i][0].repeat(B,1)
            if self.bidirectional:
                i_h, i_c = self.h0[i][1].repeat(B,1), self.c0[i][1].repeat(B,1)
            for j in range(S):
                f_out, (f_h,f_c) = layer(outputs[:,0,i,j,:].clone(), (f_h,f_c))
                outputs[:,0,i+1,j,:] = f_out

            if self.bidirectional:
                for j in reversed(range(S)):
                    i_out, (i_h,i_c) = layer(outputs[:,1,i,j,:].clone(), (i_h,i_c))
                    outputs[:,1,i+1,j,:] = i_out
                hiddens.append((torch.stack([f_h,i_h])))
                cells.append(torch.stack([f_c,i_c]))
            else:
                hiddens.append(f_h)
                cells.append(f_c)
                
        #outputs = outputs[:,:,-1:,:,:]
        return outputs[:,:,1:,:,:].contiguous(), (hiddens, cells)

In [20]:
class RNN_Encoder(nn.Module):
    def __init__(self, model = 'lstm'):
        super(RNN_Encoder, self).__init__()
        
        # No need for padding, but if we need the padding, we have to set the idx as 0
        # So that's why we need to make the graph start from 0
        self.model = model
        self.embedding = nn.Embedding(args.node_size, embedding_dim = args.embedding_dim, padding_idx=0,
                                      max_norm = args.embedding_maxnorm)
        

        if args.bidirectional:
            num_dir = 2
            fc_size1= args.hidden_size[-1]*args.seq_len*2
            fc_size2= 128
        else:
            num_dir = 1
            fc_size1= args.hidden_size[-1]
            fc_size2= 16
            
        self.fc= nn.Sequential(
            nn.Linear(fc_size1,fc_size2),
            nn.BatchNorm1d(fc_size2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(fc_size2,2),
        )
        
        
        
        if self.model == 'lstm':
            self.rnn = nn.LSTM(input_size = args.embedding_dim, hidden_size = args.hidden_size[-1],
                           num_layers = args.num_layers, bidirectional = args.bidirectional, batch_first = True)
            self.h0 = nn.Parameter(torch.randn(args.num_layers*num_dir, 1, args.hidden_size[-1]))
            self.c0 = nn.Parameter(torch.ones(args.num_layers*num_dir, 1, args.hidden_size[-1]))
            print('Using LSTM')
        elif self.model == 'gru':
            self.h0 = nn.Parameter(torch.randn(args.num_layers*num_dir, 1, args.hidden_size[-1]))
            self.rnn = nn.GRU(input_size= args.embedding_dim, hidden_size= args.hidden_size[-1],
                              num_layers = args.num_layers, bidirectional = args.bidirectional, batch_first = True)
            print('Using GRU')
            
        elif self.model == 'nalu':
            self.rnn = NALU_LSTM(args.embedding_dim, args.hidden_size, bidirectional= args.bidirectional)
            print('Using NALU')
        else:
            raise NotImplementedError

        
    def forward(self, x, lengths = None):
        
        B = x.size(0)
        
        out = self.embedding(x)
        
        if self.model == 'lstm':
            h_ = (self.h0.repeat(1,B,1), self.c0.repeat(1,B,1))
        elif self.model == 'gru':
            h_ = self.h0.repeat(1,B,1)
        
        if self.model =='nalu':
            out, hidden = self.rnn(out)
            out = out[:,:,-1,:,:].squeeze() 
        else:
            out = pack_padded_sequence(out, lengths, batch_first = True)
            out, hidden = self.rnn(out, h_)
            out, unpacked_len = pad_packed_sequence(out, batch_first=True)
        
        if args.bidirectional:
            out = self.fc(out.contiguous().view(B,-1))
        else:
            out = self.fc(out[:,-1,:])
        return out
    
    

class SiameseNet(nn.Module):
    def __init__(self,):
        super(SiameseNet, self).__init__()
        
        self.encoder = RNN_Encoder(args.RNN_model)
        self.mse = nn.MSELoss()
        self.train_hist = defaultdict(list)
        self.acc_hist = []
        self.apply(self.weight_init)
        
        self.optim = optim.Adam(self.encoder.parameters(), lr = args.lr, betas= (args.beta1, args.beta2),
                                weight_decay = args.weight_reg)
        
    def forward(self, left, left_len, right, right_len, y):
        
        self.optim.zero_grad()
#         left = x[:,0,:]
#         right = x[:,1,:]
        
        left_out = self.encoder(left, left_len)
        right_out = self.encoder(right, right_len)
        self.prediction = torch.exp(-torch.norm((left_out - right_out),1,-1))
        self.loss = self.mse(self.prediction, y)
        self.take = left_out - right_out
        
        if args.LSTM_maxnorm is not None:
            nn.utils.clip_grad_norm_(self.encoder.rnn.parameters(), args.LSTM_maxnorm)
        self.acc = torch.mean(((self.prediction>args.acc_threshold) == (y.byte())).float())
        self.train_hist['Loss'].append(self.loss.item())
        self.train_hist['Accuracy'].append(self.acc.item())
        self.loss.backward()
        self.optim.step()
        
    def weight_init(self,m):
        if type(m) in [nn.Conv2d, nn.ConvTranspose2d, nn.Linear]:
            nn.init.kaiming_normal_(m.weight,0.2,nonlinearity='leaky_relu')
        elif type(m) in [nn.LSTM]:
            for name, value in m.named_parameters():
                if 'weight' in name :
                    nn.init.xavier_normal_(value.data)
                if 'bias'in name:
                    value.data.normal_()
                    
    def model_save(self,step):
        
        path = args.model_path + args.model_name+'_Step_' + str(step) + '.pth'
        torch.save({args.model_name:self.state_dict()}, path)
        print('Model Saved')
        
    def load_step_dict(self, step):
        
        path = args.model_path + args.model_name +'_Step_' + str(step) + '.pth'
        self.load_state_dict(torch.load(path, map_location = lambda storage, loc: storage)[args.model_name])
        print('Model Loaded')
        
           
    def plot_all_loss(self, step):
        fig = plt.figure(figsize=(20, 10))
        num_loss = 2
        i = 0
        for name in self.train_hist.keys():
            if 'V' not in name:
                i+= 1
                fig.add_subplot(num_loss,1,i)
                plt.plot(self.train_hist[name], label = name)
                plt.xlabel('Number of Steps',fontsize=15)
                plt.ylabel( name, fontsize=15)
                plt.title(name, fontsize=30, fontweight ="bold")
                plt.legend(loc = 'upper left')
        plt.tight_layout()
        plt.show()
        fig.savefig("Train_Hist"+str(step)+".png") 
        
    def test_step(self, left, left_len, right, right_len, y):
        
        
        left_out = self.encoder(left, left_len)
        right_out = self.encoder(right, right_len)
        
        self.v_prediction = torch.exp(-torch.norm((left_out - right_out),1,-1)).detach()
        self.v_loss = self.mse(self.v_prediction, y)
        self.v_acc = torch.mean(((self.v_prediction>args.acc_threshold) == (y.byte())).float())
        self.train_hist['V_Loss'].append(self.v_loss.item())
        self.train_hist['V_Accuracy'].append(self.v_acc.item())    

In [21]:
train_loader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle=False, drop_last = True)
valid_loader = DataLoader(val_dataset, batch_size = args.batch_size, shuffle=False, drop_last= True)

In [22]:
args.weight_reg = 0
args.RNN_model = 'lstm'
siamese = SiameseNet().to(device)
siamese.train()
scheduler = optim.lr_scheduler.StepLR(siamese.optim, 2000, gamma=0.7)
siamese.optim.param_groups[0]['lr']= 0.0002

Using LSTM


In [23]:
train_hist = defaultdict(list)
args.running_loss = True
args.n_epoch = 50
all_step = 0
epoch = 0

In [24]:
while epoch < args.n_epoch:
    siamese.train()
    for i, (left, left_len, right, right_len, y) in enumerate(train_loader):
        
        
        scheduler.step()
        start_t = time.time()
        
        left = left.to(device)
        left_len = left_len.to(device)
        right = right.to(device)
        right_len = right_len.to(device)
        y = y.to(device)
        
        siamese(left, left_len, right, right_len, y)
        end_t = time.time()
        all_step += 1
        print('| Epoch [%d] | Step [%d] | lr [%.6f] | Loss: [%.4f] | Acc: [%.4f] | Time: %.1fs' %\
              ( epoch, all_step, siamese.optim.param_groups[0]['lr'], siamese.loss.item() , siamese.acc.item() ,
                end_t - start_t))
        
#         if siamese.acc.item() > 0.95: # Stop early
#             raise StopIteration
    
    if i % 1 == 0:
        siamese.eval()
        for j, (v_left, v_left_len, v_right, v_right_len, v_y) in enumerate(valid_loader):

            v_left = v_left.to(device)
            v_left_len = v_left_len.to(device)
            v_right = v_right.to(device)
            v_right_len = v_right_len.to(device)
            v_y = v_y.to(device)
            siamese.test_step(v_left, v_left_len, v_right, v_right_len, v_y)
            end_t = time.time()
            print('| Epoch [%d] | Validation | Step [%d] |  Loss: [%.4f] | Acc: [%.4f] | Time: %.1fs' %\
                  ( epoch, j, siamese.v_loss.item() , siamese.v_acc.item() ,end_t - start_t))
                
    siamese.plot_all_loss('S_LSTMCycle_Training_0reg')
    
    if epoch >= 1:
        plot_train_hist(train_hist, 'S_LSTMCycle_Epoch_0reg')

    for name in siamese.train_hist.keys():
        train_hist[name].append(sum(siamese.train_hist[name])/len(siamese.train_hist[name]))

    if not args.running_loss:
        for name in siamese.train_hist.keys():
            siamese.train_hist[name] = []
            
    epoch += 1
    
    if epoch % 10 == 0 and epoch != 0:
        siamese.model_save(epoch)
    
    if epoch >= 20:
        raise StopIteration

| Epoch [0] | Step [1] | lr [0.020000] | Loss: [0.3605] | Acc: [0.6250] | Time: 1.5s
| Epoch [0] | Step [2] | lr [0.020000] | Loss: [0.3563] | Acc: [0.5312] | Time: 1.5s
| Epoch [0] | Step [3] | lr [0.020000] | Loss: [0.2949] | Acc: [0.6250] | Time: 1.6s


KeyboardInterrupt: 

In [ ]:
##### This Algorithm can solve the problem and make the graph in a batch of training data #####

class Graph_Alg(object):
    
    def __init__(self,):
        self.G = nx.Graph()
    
    def forward(self, x, y):
        
        paired_data = np.array(x[y.byte()])
        for i in paired_data:
            for j in i:
                for k in range(len(j)):
                    if not j[k] in self.G.nodes:
                        self.G.add_node(j[k])
                    if k >= 1:
                        if not (j[k],j[k-1]) in self.G.edges:
                            self.G.add_edge(j[k],j[k-1])
                            
    def draw(self,):
        
        nx.draw(self.G, with_labels=True, font_weight='bold')
        plt.show()
        
    def test(self, x, y):
        
        out = []
        paired_data = np.array(x)
        
        for i in paired_data:
            out.append(self.check_step(i))
        match = list(np.array(out) == np.array(y).astype(int))
        self.acc = sum(match) / len(match)
             
    def check_step(self, i):
        
        # Check the path is in the graph
        for j in i:
            for k in range(len(j)):
                if not j[k] in self.G.nodes:
                    return 0
                if k >= 1:
                    if not (j[k],j[k-1]) in self.G.edges:
                        return 0
                        
        # Check the the pair is possible to have the same st_node   
        for node in list(self.G.adj[i[0][0].item()]):
            if node in list(self.G.adj[i[1][0].item()]):
                return 1

        return 0


In [ ]:
graph_algo = Graph_Alg()
algo_train_hist = defaultdict(list)
num_steps = 20
for i in range(num_steps):
    data, label = iter(train_loader).next()

    # Training
    graph_algo.forward(data, label)
    
    # Testing
    v_data, v_label = iter(valid_loader).next()
    graph_algo.test(v_data, v_label)
    algo_train_hist['Accuracy'].append(graph_algo.acc)
    
plot_train_hist(algo_train_hist, 'Graph_algo')
graph_algo.draw()